[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1lSiBqgLskDVZqn7TdEPzP6CYA2iP9RdB)

Uncomment the following line to install [geoca](https://haorui-jiang.github.io/geoca/) if needed.

In [ ]:
# !pip install geoca

Import two modules, **rasterconvert** and **cellularautomata**.

Use **rasterconvert** for converting raster data formats and **cellularautomata** for running CA models.

In [ ]:
from geoca import rasterconvert, cellularautomata

**rasterconvert.raster_list(file_path)**:

- Reads raster data from a file and converts the data to a Python list where the NoData value is converted to None.

In [ ]:
raster_file = "Data/raster.tif"
data_list = rasterconvert.raster_list(raster_file)

# Read the first 100 rows of a data list and output all non-None values
for row in data_list[:100]:
    non_none_values = [value for value in row if value is not None]
    if non_none_values:
        print(non_none_values)

In [ ]:
# Set the file paths for the original raster data and the new raster data to be output
input_raster_path = raster_file
output_raster_path = "Data/result.tif"

**cellularautomata.migrate_population_focus(data_list, population, direction_num=4, proportion=1)**:

- The population is **focused towards** the most suitable nearby migration areas based on the raster pixel values.

**cellularautomata.migrate_population_disperse(data_list, population, direction_num=4, proportion=[0.5, 0.25, 0.15, 0.05])**:

- The population is **dispersed** and migrates to the neighborhood based on the raster pixel values.

**rasterconvert.process_raster_data(input_raster_path, output_raster_path, new_data, nodata_value)**:

- Create a raster template based on the input raster, process the raster data by replacing pixel values with the new data.

The **migrate_population_focus()** or **migrate_population_disperse()** function can be used when exporting raster data after completing some specific rounds of iterations.

**Note:** When using the **migrate_population_disperse()** function, it is not recommended to iterate too many times as the population migrates to multiple neighboring locations, which may result in an overly dispersed population.

In [ ]:
# Run the CA model for a specific number of iterations with an initial population size of 10 per pixel

# Create a list of regional populations of the same size as the raster data list and uniformly set all list elements to a value of 10, representing the initial population size of each raster pixel
population = [[10 for _ in range(len(data_list[0]))] for _ in range(len(data_list))]

iterations = 20  # Set the number of iterations

for i in range(iterations):
    # Migration of 60% of the population to the most suitable nearby raster pixel.
    population = cellularautomata.migrate_population_focus(data_list, population, 4, 0.6)

    # The suitability of the neighboring raster pixels is sorted,
    # with 1/2 of the population moving in the direction of the maximum,
    # 1/4 of the population moving in the second largest direction,
    # 1/8 of the population moving in the third largest direction,
    # and the rest of the population remaining where they are.
    population = cellularautomata.migrate_population_disperse(data_list, population, 4, [0.5, 0.25, 0.125, 0])
    print(f"Iteration {i + 1} is complete.")

    # When the number of iterations is a multiple of 10, output the iteration results as raster data
    if (i + 1) % 10 == 0:
        # After process_raster_data() finishes running, the NoData value is 0
        rasterconvert.process_raster_data(input_raster_path, f"Data/iterations_{i + 1}.tif", population, 0)

To export raster data after all iterations have been completed, use the **run_iterations_num()** function or the **run_iterations_pop()** function.

**cellularautomata.run_iterations_num(iterations, data_list, population_num=10, direction_num=4, type_migration="focus", migration_proportion=1)**:

- Running a cellular automata using **a uniform initial population count** to simulate population migration based on a raster of environmental data.

In [ ]:
# Run the cellular automata simulation for 5 iterations with an initial population count of 10 at each pixel
new_data = cellularautomata.run_iterations_num(5, data_list, population_num=10)

rasterconvert.process_raster_data(input_raster_path, output_raster_path, new_data, 0)

**cellularautomata.run_iterations_pop(iterations, data_list, population_list, direction_num=4, type_migration="focus", migration_proportion=1)**:

- Running a cellular automata using **an initial population size raster** to simulate population migration based on a raster of environmental data.

In [ ]:
population_file = "Data/population.tif"
population_list = rasterconvert.raster_list(population_file)

# Run the cellular automata simulation for 5 iterations, with the initial population size of each pixel determined from a list of population raster values
new_data = cellularautomata.run_iterations_pop(5, data_list, population_list)

rasterconvert.process_raster_data(input_raster_path, output_raster_path, new_data, 0)

**cellularautomata.migrate_time(data_list, cost_list)**:

- Calculate the migration time based on the cost path raster and the environment raster.

In [ ]:
cost_file = "Data/CostPath.tif"
cost_list = rasterconvert.raster_list(cost_file)

# Output the migration time, number of iterations,
# and a list of environmental raster values corresponding to the cost paths
time, count, costData_list = cellularautomata.migrate_time(data_list, cost_list)
print(time, count)
print(costData_list, len(costData_list))